In [1]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
raw_data_path=os.path.join('data','raw')
df=pd.read_csv(os.path.join(raw_data_path,'fer2013.csv'))
X=[]
for i in range(len(df)):
    X.append(df.loc[i,'pixels'].split(' '))
X=np.array(X,np.float32)/255.
y=df.emotion.values
X_train=X[np.where(df.Usage=='Training')]
y_train=y[np.where((df.Usage=='Training'))]
X_test=X[np.where((df.Usage!='Training'))]
y_test=y[np.where((df.Usage!='Training'))]
def balance_classes(X,y):
    X1=X[np.where(y!=1)]
    X2=X[np.where(y==1)]
    X2=np.repeat(X2,9,axis=0)
    X=np.concatenate((X1,X2),axis=0)
    y1=y[np.where(y!=1)]
    y2=y[np.where(y==1)]
    y2=np.repeat(y2,9,axis=0)
    y=np.concatenate((y1,y2),axis=0)
    i=[i for i in range(len(y))]
    np.random.shuffle(i)
    return X[i],y[i]
X_train,y_train=balance_classes(X_train,y_train)

def train_test_split(X,y,size=0.8,rs=None):
    N=len(X)
    np.random.seed(rs)
    tr_i=np.random.choice(N,int(size*N),replace=False)
    t_i=[i for i in range(N) if i not in tr_i]
    X_train=X[tr_i]
    y_train=y[tr_i]
    X_test=X[t_i]
    y_test=y[t_i]
    return X_train,y_train,X_test,y_test
# X_train,y_train,X_test,y_test=train_test_split(X_s,y,size=0.9,rs=20)

In [13]:
X_train=X_train.astype(np.float32)
y_train=y_train.astype(np.float32)
X_test=X_test.astype(np.float32)
y_test=y_test.astype(np.float32)

In [51]:
def one_hot_encoder(X):
    N=X.shape[0]
    K=len(set(X))
    Z=np.zeros((N,K))
    for i in range(N):
        c=int(X[i])
        Z[i,c]=1
    return Z

In [42]:
one_hot_encoder(y_train)

(32197, 7)


array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 1., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 1.]])

In [36]:
import tensorflow as tf

In [19]:
class HiddenLayer(object):
    def __init__(self,i,M1,M2,af='relu',rs=None):
        self.id=i
        self.M1=M1
        self.M2=M2
        self.af=af
        
        self.W=tf.Variable(tf.random_normal(shape=[M1,M2],stddev=np.sqrt(M1),seed=rs,dtype=tf.float32))
        self.b=tf.Variable(tf.random_normal(shape=[M2],seed=rs,dtype=tf.float32))
        self.params=[self.W,self.b]
    def forward(self,X):
        if self.af=='tanh':
            return tf.nn.tanh(tf.matmul(X,self.W)+self.b)
        if self.af=='sigmoid':
            return tf.nn.sigmoid(tf.matmul(X,self.W)+self.b)
        if self.af=='relu':
            return tf.nn.relu(tf.matmul(X,self.W)+self.b)
        if self.af=='leaky-relu':
            return tf.nn.leaky_relu(tf.matmul(X,self.W)+self.b)

In [63]:
class ANN(object):
    def __init__(self,hidden_layer_size):
        self.hidden_layer_size=hidden_layer_size
        
    def forward(self,X):
        z=X
        for h in self.hidden_layers:
            z=h.forward(z)
        return tf.matmul(z,self.W)+self.b
    
    def predict(self,X):
        p=self.forward(X)
        return tf.argmax(p,axis=1)
    
    def fit(self,X,y,lr=0.001,epochs=1000,af=None,fig=True,batch_size=2900,n_batch=10,random_state=None):
        y=one_hot_encoder(y)
        if af==None:
            af=len(self.hidden_layer_size)*['relu']
        X_t,y_t,X_v,y_v=train_test_split(X,y,size=0.9,rs=random_state)
        c_t=[]
        c_v=[]
        cl_t=[]
        cl_v=[]
        N,D=X_t.shape
        self.hidden_layers=[]
        M1=D
        cnt=0
        for M2,a in zip(self.hidden_layer_size,af):
            h=HiddenLayer(cnt,M1,M2,a,random_state)
            self.hidden_layers.append(h)
            M1=M2
            cnt+=1
        M2=y.shape[1]
        self.W=tf.Variable(tf.random_normal([M1,M2],stddev=np.sqrt(M1),seed=random_state,dtype=tf.float32))
        self.b=tf.Variable(tf.random_normal(shape=[M2],seed=random_state,dtype=tf.float32))
        self.params=[self.W,self.b]
        for h in self.hidden_layers:
            self.params+=h.params
        tfX=tf.placeholder(tf.float32,shape=[None,D])
        tfY=tf.placeholder(tf.float32,shape=[None,M2])
        y_p=self.forward(tfX)
        cost=tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=y_p,labels=tfY))
        preds=self.predict(tfX)
        optimizer=tf.train.RMSPropOptimizer(lr)
        train=optimizer.minimize(cost)
        with tf.Session() as s:
            s.run(tf.global_variables_initializer())
            for i in range(epochs):
                for j in range(n_batch):
                    x=X_t[j*batch_size:(j*batch_size+batch_size)]
                    yb=y_t[j*batch_size:j*batch_size+batch_size]
                    c_t.append(s.run(cost,feed_dict={tfX:x,tfY:yb}))
                    c_v.append(s.run(cost,feed_dict={tfX:X_v,tfY:y_v}))
                    cl_t.append(np.mean(s.run(preds,feed_dict={tfX:x})==np.argmax(yb,axis=1)))               
                    cl_v.append(np.mean(s.run(preds,feed_dict={tfX:X_v})==np.argmax(y_v,axis=1)))    
                    s.run(train,feed_dict={tfX:x,tfY:yb})
                if i%10==0:
                    print('Epoch {0} Train C: {1} Cl: {2} Test C: {3} Cl:{4}'.format(i,c_t[i],cl_t[i],c_v[i],cl_v[i]))
        if fig==True:
            plt.plot(c_t,label='Train Cost')
            plt.plot(c_v,label='Test Cost')
            plt.legend()
            plt.show()
            plt.plot(cl_t,label='Train Classification')
            plt.plot(cl_v,label='Test Classification')
            plt.legend()
            plt.show()

In [64]:
a=ANN((10,20,30,5,10,50,30))

In [ ]:
a.fit(X_train,y_train,lr=0.0001,epochs=10000)

Epoch 0 Train C: 86320177152.0 Cl: 0.14517241379310344 Test C: 86605242368.0 Cl:0.15
Epoch 10 Train C: 84853022720.0 Cl: 0.14413793103448275 Test C: 85137424384.0 Cl:0.15
Epoch 20 Train C: 83911434240.0 Cl: 0.14344827586206896 Test C: 84196777984.0 Cl:0.1496894409937888
Epoch 30 Train C: 83049218048.0 Cl: 0.14344827586206896 Test C: 83335274496.0 Cl:0.14937888198757765
Epoch 40 Train C: 82212782080.0 Cl: 0.14275862068965517 Test C: 82499878912.0 Cl:0.1484472049689441
Epoch 50 Train C: 81389731840.0 Cl: 0.1424137931034483 Test C: 81677516800.0 Cl:0.1484472049689441
Epoch 60 Train C: 80574717952.0 Cl: 0.1424137931034483 Test C: 80862183424.0 Cl:0.1484472049689441
Epoch 70 Train C: 79766437888.0 Cl: 0.1424137931034483 Test C: 80053116928.0 Cl:0.14906832298136646
Epoch 80 Train C: 78964121600.0 Cl: 0.14172413793103447 Test C: 79250759680.0 Cl:0.14875776397515528
Epoch 90 Train C: 78168219648.0 Cl: 0.14206896551724138 Test C: 78455316480.0 Cl:0.14813664596273293
Epoch 100 Train C: 773791662

Epoch 830 Train C: 34402582528.0 Cl: 0.14275862068965517 Test C: 34307299328.0 Cl:0.1406832298136646
Epoch 840 Train C: 34011156480.0 Cl: 0.14310344827586208 Test C: 33915318272.0 Cl:0.1409937888198758
Epoch 850 Train C: 33624936448.0 Cl: 0.1413793103448276 Test C: 33527322624.0 Cl:0.1422360248447205
Epoch 860 Train C: 33242402816.0 Cl: 0.14 Test C: 33144604672.0 Cl:0.14161490683229813
Epoch 870 Train C: 32866635776.0 Cl: 0.1393103448275862 Test C: 32766193664.0 Cl:0.1422360248447205
Epoch 880 Train C: 32496871424.0 Cl: 0.1393103448275862 Test C: 32393138176.0 Cl:0.14130434782608695
Epoch 890 Train C: 32134946816.0 Cl: 0.1406896551724138 Test C: 32027185152.0 Cl:0.1409937888198758
Epoch 900 Train C: 31777777664.0 Cl: 0.14103448275862068 Test C: 31666882560.0 Cl:0.14037267080745341
Epoch 910 Train C: 31422840832.0 Cl: 0.1413793103448276 Test C: 31308926976.0 Cl:0.1409937888198758
Epoch 920 Train C: 31072225280.0 Cl: 0.14206896551724138 Test C: 30954864640.0 Cl:0.14130434782608695
Epoch 

Epoch 1640 Train C: 14616550400.0 Cl: 0.1296551724137931 Test C: 14277740544.0 Cl:0.13012422360248446
Epoch 1650 Train C: 14464852992.0 Cl: 0.12931034482758622 Test C: 14124375040.0 Cl:0.13198757763975155
Epoch 1660 Train C: 14313863168.0 Cl: 0.12931034482758622 Test C: 13973693440.0 Cl:0.13105590062111802
Epoch 1670 Train C: 14163525632.0 Cl: 0.13 Test C: 13824269312.0 Cl:0.13136645962732918
Epoch 1680 Train C: 14012717056.0 Cl: 0.1296551724137931 Test C: 13675546624.0 Cl:0.13167701863354037
Epoch 1690 Train C: 13862122496.0 Cl: 0.1306896551724138 Test C: 13528870912.0 Cl:0.13260869565217392
Epoch 1700 Train C: 13712316416.0 Cl: 0.1317241379310345 Test C: 13384169472.0 Cl:0.13509316770186336
Epoch 1710 Train C: 13564210176.0 Cl: 0.13241379310344828 Test C: 13240450048.0 Cl:0.134472049689441
Epoch 1720 Train C: 13416986624.0 Cl: 0.13310344827586207 Test C: 13098522624.0 Cl:0.13291925465838508
Epoch 1730 Train C: 13270497280.0 Cl: 0.13275862068965516 Test C: 12957702144.0 Cl:0.132608695

Epoch 2470 Train C: 5319372288.0 Cl: 0.14793103448275863 Test C: 5204617216.0 Cl:0.16055900621118013
Epoch 2480 Train C: 5240525312.0 Cl: 0.14862068965517242 Test C: 5126397440.0 Cl:0.15962732919254657
Epoch 2490 Train C: 5162201600.0 Cl: 0.14862068965517242 Test C: 5049061376.0 Cl:0.15931677018633542
Epoch 2500 Train C: 5084778496.0 Cl: 0.15 Test C: 4972450304.0 Cl:0.15931677018633542
Epoch 2510 Train C: 5007972352.0 Cl: 0.15 Test C: 4895965696.0 Cl:0.1577639751552795
Epoch 2520 Train C: 4931494400.0 Cl: 0.1493103448275862 Test C: 4819928064.0 Cl:0.1565217391304348
Epoch 2530 Train C: 4855052800.0 Cl: 0.1493103448275862 Test C: 4743672320.0 Cl:0.15683229813664595
Epoch 2540 Train C: 4778899456.0 Cl: 0.1493103448275862 Test C: 4667438592.0 Cl:0.15745341614906833
Epoch 2550 Train C: 4703329792.0 Cl: 0.13275862068965516 Test C: 4591834112.0 Cl:0.134472049689441
Epoch 2560 Train C: 4627899392.0 Cl: 0.13241379310344828 Test C: 4516576256.0 Cl:0.13260869565217392
Epoch 2570 Train C: 4552390

Epoch 3300 Train C: 915442624.0 Cl: 0.14586206896551723 Test C: 855298112.0 Cl:0.1422360248447205
Epoch 3310 Train C: 891689344.0 Cl: 0.1493103448275862 Test C: 833520384.0 Cl:0.14906832298136646
Epoch 3320 Train C: 868658432.0 Cl: 0.1424137931034483 Test C: 812393728.0 Cl:0.1425465838509317
Epoch 3330 Train C: 846065792.0 Cl: 0.14379310344827587 Test C: 792022016.0 Cl:0.14409937888198757
Epoch 3340 Train C: 824259392.0 Cl: 0.1496551724137931 Test C: 772570432.0 Cl:0.14906832298136646
Epoch 3350 Train C: 802881280.0 Cl: 0.1496551724137931 Test C: 753485760.0 Cl:0.14875776397515528
Epoch 3360 Train C: 781855808.0 Cl: 0.15 Test C: 734600768.0 Cl:0.14130434782608695
Epoch 3370 Train C: 761239040.0 Cl: 0.1503448275862069 Test C: 716169728.0 Cl:0.1406832298136646
Epoch 3380 Train C: 741268416.0 Cl: 0.14655172413793102 Test C: 698265024.0 Cl:0.1425465838509317
Epoch 3390 Train C: 722388928.0 Cl: 0.14275862068965517 Test C: 681118272.0 Cl:0.14285714285714285
Epoch 3400 Train C: 703728128.0 Cl

Epoch 4150 Train C: 106420248.0 Cl: 0.1510344827586207 Test C: 109139968.0 Cl:0.1512422360248447
Epoch 4160 Train C: 103813160.0 Cl: 0.1520689655172414 Test C: 106702136.0 Cl:0.15248447204968943
Epoch 4170 Train C: 101253976.0 Cl: 0.15172413793103448 Test C: 104379184.0 Cl:0.15279503105590062
Epoch 4180 Train C: 98780176.0 Cl: 0.15241379310344827 Test C: 102143704.0 Cl:0.153416149068323
Epoch 4190 Train C: 96435480.0 Cl: 0.1520689655172414 Test C: 99997248.0 Cl:0.1531055900621118
Epoch 4200 Train C: 94185392.0 Cl: 0.1510344827586207 Test C: 97889864.0 Cl:0.1515527950310559
Epoch 4210 Train C: 92085560.0 Cl: 0.1520689655172414 Test C: 95837544.0 Cl:0.14906832298136646
Epoch 4220 Train C: 90038360.0 Cl: 0.15068965517241378 Test C: 93780712.0 Cl:0.1515527950310559
Epoch 4230 Train C: 88268808.0 Cl: 0.1503448275862069 Test C: 91832256.0 Cl:0.1512422360248447
Epoch 4240 Train C: 86524512.0 Cl: 0.1482758620689655 Test C: 89891016.0 Cl:0.1515527950310559
Epoch 4250 Train C: 84773816.0 Cl: 0.1

array([3., 3., 3., ..., 1., 0., 6.], dtype=float32)